### Granger causality Part II: The Movie
This is a follow-up notebook to ["*Granger causality testing for 1 day*"](https://www.kaggle.com/carlmcbrideellis/granger-causality-testing-for-1-day). The cryptocurrencies on the X-axis are the *leaders*, and on the Y-axis the *followers*. The greater the value, the more they follow. Note that I pre-set the maximum value of the colour scale to be 100 to better allow comparison between days. However, that is not the global maximum value, which for the dates examined below corresponds to the 7th of September 2021 with a great many tests with high values (one reaching a value of over 290).
### Just press play...

In [ ]:
%matplotlib inline
from matplotlib import rc
from IPython.display import HTML
import matplotlib.animation as animation

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 14})
#plt.rcParams["figure.figsize"] = (8, 8)
import seaborn as sns
import plotly.express as px

import datetime
from datetime import timedelta, date

import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests

# read in the data
train = pd.read_csv("../input/g-research-crypto-forecasting/train.csv")
asset_details = pd.read_csv("../input/g-research-crypto-forecasting/asset_details.csv")
train['timestamp'] = pd.to_datetime(train['timestamp'], unit='s')
train = train.set_index('timestamp')
mapping = dict(asset_details[['Asset_ID', 'Asset_Name']].values)
train["Asset_name"] = train["Asset_ID"].map(mapping)
cryptocurrencies = asset_details['Asset_Name'].unique()

granger_matrix = np.zeros((14, 14), float)

In [ ]:
first_day  = datetime.datetime(2021, 5, 1, 0, 0, 0)  # start from midnight the 1st of May 2021
delta_time = datetime.timedelta(days=1) # look at days
n_steps    = 143  # look at the 143 days up to the 20th of September 2021 
lag        =   1  # look at the 1 minute lag
colour_max = 100  # set the upper limit of the colour scale 

In [ ]:
%%capture
fig, (ax) = plt.subplots(1, figsize = (11, 11));

def animate(time_stepper):
    start_time = first_day  + (time_stepper * delta_time)
    end_time   = start_time + delta_time
    # now extract this time window from the training data
    last_day   = train.loc[start_time:end_time]
    # create columns of the Close values
    close_columns = pd.pivot_table(last_day,index='timestamp',columns='Asset_ID',values='Close')
    for i in range(14):
        for j in range(14):
            results = grangercausalitytests(close_columns[[i, j]].pct_change().dropna(), [lag], verbose=False)
            granger_matrix[i,j] = results[lag][0]['ssr_ftest'][0]
    ax.cla() # clears the annotations rather than overwriting
    sns.heatmap(ax = ax, data = granger_matrix,cmap='YlOrBr',annot=True, fmt='.1f',cbar=False, vmin=0, vmax=colour_max)
    plt.title(start_time.strftime("%-d %B %Y") +" to "+ end_time.strftime("%-d %B %Y"))
    plt.yticks(asset_details.Asset_ID.values +0.5, asset_details.Asset_Name.values, rotation='horizontal');
    plt.xticks(asset_details.Asset_ID.values +0.5, asset_details.Asset_Name.values, rotation='vertical');

anim = animation.FuncAnimation(fig, animate, frames=n_steps, save_count=n_steps, repeat=False)
HTML(anim.to_jshtml())

In [ ]:
HTML(anim.to_jshtml())